# Getting model "cast" files into one dataframe with depth matched to observations

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import gsw
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

In [2]:
def nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [3]:
path = Path("/data1/bbeutel/LO_output/extract_cast/")

## NOTE: despite units being g/kg, the model salinity is practical salinity!
### source: https://github.com/parkermac/LPM/blob/main/obsmod/process_multi_ctd.py line 103

In [4]:
def read_dfo_ctd(year):    

    dfo = pd.read_pickle('/data1/parker/LO_output/obs/dfo1/ctd/'+str(year)+'.p')

    dfo['h'] = np.nan
    dfo['z_model'] = np.nan
    dfo['SA_model'] = np.nan
    dfo['CT_model'] = np.nan
    dfo['DO_model'] = np.nan
    cid = np.arange(0,int(np.max(dfo.cid))+1,1)

    files = [sorted(path.glob("dfo1_ctd_{}/{}.nc".format(year,str(cid[i])))) for i in range(len(cid))]

    for i in range(len(files)):
        if len(files[i]) == 1:
            # print(i)
            cast = xr.open_dataset(files[i][0])

            dfo.h[dfo.cid == i] = cast.h
            dfo.z_model[dfo.cid == i] = np.array([nearest(cast.h*cast.s_rho,dfo.z[dfo.cid == i][j]) 
                                                        for j in np.array(dfo[dfo.cid == i].index)])
            PT = np.array([cast.temp[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])
            SP = np.array([cast.salt[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])
            dfo.DO_model[dfo.cid == i] = np.array([cast.oxygen[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])

            dfo.SA_model[dfo.cid == i] = gsw.SA_from_SP(SP, np.array(dfo.z[dfo.cid == i]*-1), np.array(dfo.lon[dfo.cid == i]), np.array(dfo.lat[dfo.cid == i]))
            dfo.CT_model[dfo.cid == i] = gsw.CT_from_pt(dfo.SA_model[dfo.cid == i], PT)

    name = "dfo_ctd_"+str(year)+".csv"
    dfo.to_csv(name)

In [35]:
# read_dfo_ctd(2018)

In [10]:
def read_dfo_bottle(year):    

    dfo = pd.read_pickle('/data1/parker/LO_output/obs/dfo1/bottle/'+str(year)+'.p').drop(axis=1,labels=['name','cruise']).dropna()

    dfo['h'] = np.nan
    dfo['z_model'] = np.nan
    dfo['SA_model'] = np.nan
    dfo['CT_model'] = np.nan
    dfo['DO_model'] = np.nan
    dfo['NO3_model'] = np.nan

    cid = np.arange(0,int(np.max(dfo.cid))+1,1)

    files = [sorted(path.glob("dfo1_bottle_{}/{}.nc".format(year,str(cid[i])))) for i in range(len(cid))]

    for i in range(len(files)):
        if len(files[i]) == 1:
            cast = xr.open_dataset(files[i][0])


            dfo.h[dfo.cid == i] = cast.h
            dfo.z_model[dfo.cid == i] = np.array([nearest(cast.h*cast.s_rho,dfo.z[dfo.cid == i][j]) 
                                                        for j in np.array(dfo[dfo.cid == i].index)])
            PT = np.array([cast.temp[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])
            SP = np.array([cast.salt[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])
            dfo.DO_model[dfo.cid == i] = np.array([cast.oxygen[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])
            dfo.NO3_model[dfo.cid == i] = np.array([cast.NO3[int(dfo.z_model[j])] for j in np.array(dfo[dfo.cid == i].index)])
            # dfo.Chl_model[dfo.cid == i] = np.array([cast.oxygen[int(dfo.z_model)]])
                    # dont really know what to do with Chl yet but might as well take it


            dfo.SA_model[dfo.cid == i] = gsw.SA_from_SP(SP, np.array(dfo.z[dfo.cid == i]*-1), np.array(dfo.lon[dfo.cid == i]), np.array(dfo.lat[dfo.cid == i]))
            dfo.CT_model[dfo.cid == i] = gsw.CT_from_pt(dfo.SA_model[dfo.cid == i], PT)

    name = "dfo_botte_"+str(year)+".csv"
    dfo.to_csv(name)

In [11]:
read_dfo_bottle(2019)

In [12]:
read_dfo_bottle(2021)

In [13]:
read_dfo_bottle(2018)

In [14]:
read_dfo_bottle(2020)

In [23]:
pd.read_csv('dfo_botte_2017.csv').head()

,Unnamed: 0,cid,lon,lat,time,z,SA,CT,DO (uM),NO3 (uM),Chl (mg m-3),h,z_model,SA_model,CT_model,DO_model,NO3_model
0,0,0.0,-126.3340,48.624500,2017-02-07 14:50:45,-5.9,32.698785,9.202596,282.23343,5.9,0.57,735.887373,29.0,32.862832,9.055119,270.132965,13.084056
1,46,3.0,-126.6655,48.651333,2017-02-07 22:43:09,-1.5,32.438786,9.175869,284.46628,5.3,0.87,1307.365892,29.0,32.783173,9.077599,271.404907,12.221288
2,49,3.0,-126.6655,48.651333,2017-02-07 22:43:09,-5.9,32.493155,9.195749,284.46628,5.4,0.73,1307.365892,29.0,32.783104,9.077600,271.404907,12.221288
3,50,3.0,-126.6655,48.651333,2017-02-07 22:43:09,-10.0,32.512463,9.170014,284.46628,5.4,0.82,1307.365892,29.0,32.783039,9.077601,271.404907,12.221288
4,51,3.0,-126.6655,48.651333,2017-02-07 22:43:09,-14.7,32.516979,9.159109,284.01970,5.5,0.70,1307.365892,29.0,32.783026,9.077601,271.404907,12.221288


In [7]:
def read_ncei(CoastalSalish, year):    

    d = pd.read_pickle('/data1/parker/LO_output/obs/ncei'+CoastalSalish+'/bottle/'+str(year)+'.p').drop(axis=1,labels=['name','cruise'])#.dropna()
    d['h'] = np.nan
    d['z_model'] = np.nan
    d['SA_model'] = np.nan
    d['CT_model'] = np.nan
    d['DO_model'] = np.nan
    d['NO3_model'] = np.nan
    d['TIC_model'] = np.nan
    d['TA_model'] = np.nan


    cid = np.arange(0,int(np.max(d.cid))+1,1)

    files = [sorted(path.glob("ncei"+CoastalSalish+"_bottle_{}/{}.nc".format(year,str(cid[i])))) for i in range(len(cid))]

    for i in range(len(files)):
        if len(files[i]) == 1:
            # print(files[i][0])
            cast = xr.open_dataset(files[i][0])


            d.h[d.cid == i] = cast.h
            d.z_model[d.cid == i] = np.array([nearest(cast.h*cast.s_rho,d.z[d.cid == i][j]) 
                                                        for j in np.array(d[d.cid == i].index)])
            PT = np.array([cast.temp[int(d.z_model[j])] for j in np.array(d[d.cid == i].index)])
            SP = np.array([cast.salt[int(d.z_model[j])] for j in np.array(d[d.cid == i].index)])
            d.DO_model[d.cid == i] = np.array([cast.oxygen[int(d.z_model[j])] for j in np.array(d[d.cid == i].index)])
            d.NO3_model[d.cid == i] = np.array([cast.NO3[int(d.z_model[j])] for j in np.array(d[d.cid == i].index)])
            d.TIC_model[d.cid == i] = np.array([cast.TIC[int(d.z_model[j])] for j in np.array(d[d.cid == i].index)])
            d.TA_model[d.cid == i] = np.array([cast.alkalinity[int(d.z_model[j])] for j in np.array(d[d.cid == i].index)])

    
            d.SA_model[d.cid == i] = gsw.SA_from_SP(np.array(SP), np.array(d.z[d.cid == i]*-1), np.array(d.lon[d.cid == i]), np.array(d.lat[d.cid == i]))
            d.CT_model[d.cid == i] = gsw.CT_from_pt(d.SA_model[d.cid == i], PT)

    name = "ncei"+CoastalSalish+"_"+str(year)+".csv"
    d.to_csv(name)